##### Copyright 2020 Les auteurs de TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction aux dégradés et à la différenciation automatique

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Voir sur TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Exécuter dans Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Afficher la source sur GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Télécharger le cahier</a></td>
</table>

## Différenciation et dégradés automatiques

[La différenciation automatique](https://en.wikipedia.org/wiki/Automatic_differentiation) est utile pour mettre en œuvre des algorithmes d'apprentissage automatique tels que la [rétropropagation](https://en.wikipedia.org/wiki/Backpropagation) pour la formation des réseaux de neurones.

Dans ce guide, nous discuterons des façons dont vous pouvez calculer des dégradés avec TensorFlow, en particulier lors d'une exécution rapide.

## Installer

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

## Calcul des gradients

Pour se différencier automatiquement, TensorFlow doit se rappeler quelles opérations se produisent dans quel ordre pendant la passe *avant* . Ensuite, lors du *passage en arrière* , TensorFlow parcourt cette liste d'opérations dans l'ordre inverse pour calculer les gradients.

## Bandes dégradées

TensorFlow fournit l'API [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) pour la différenciation automatique; c'est-à-dire, calculer le gradient d'un calcul par rapport à certaines entrées, généralement `tf.Variable` s. TensorFlow "enregistre" les opérations pertinentes exécutées dans le contexte d'un `tf.GradientTape` sur une "bande". TensorFlow utilise ensuite cette bande pour calculer les gradients d'un calcul "enregistré" en utilisant [la différenciation en mode inverse](https://en.wikipedia.org/wiki/Automatic_differentiation) .

Voici un exemple simple:

In [ ]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
  y = x**2

Une fois que vous avez enregistré certaines opérations, utilisez `GradientTape.gradient(target, sources)` pour calculer le gradient d'une cible (souvent une perte) par rapport à une source (souvent les variables du modèle).

In [ ]:
# dy = 2x * dx
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

L'exemple ci-dessus utilise des scalaires, mais `tf.GradientTape` fonctionne aussi facilement sur n'importe quel tenseur:

In [ ]:
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)

Pour obtenir le gradient de `y` par rapport aux deux variables, vous pouvez passer les deux comme sources à la méthode du `gradient` . La bande est flexible sur la façon dont les sources sont transmises et acceptera toute combinaison imbriquée de listes ou de dictionnaires et retournera le dégradé structuré de la même manière (voir `tf.nest` ).

In [ ]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

Le dégradé par rapport à chaque source a la forme de la source:

In [ ]:
print(w.shape)
print(dl_dw.shape)

Voici à nouveau le calcul du gradient, cette fois en passant un dictionnaire de variables:

In [ ]:
my_vars = {
    'w': tf.Variable(tf.random.normal((3, 2)), name='w'),
    'b': tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
}

grad = tape.gradient(loss, my_vars)
grad['b']

## Dégradés par rapport à un modèle

It's common to collect `tf.Variables` into a `tf.Module` or one of its subclasses (`layers.Layer`, `keras.Model`) for [checkpointing](checkpoint.ipynb) and [exporting](saved_model.ipynb).

Dans la plupart des cas, vous souhaiterez calculer des gradients par rapport aux variables entraînables d'un modèle. Étant donné que toutes les sous-classes de `tf.Module` agrègent leurs variables dans la propriété `Module.trainable_variables` , vous pouvez calculer ces dégradés en quelques lignes de code: 

In [ ]:
layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
  # Forward pass
  y = layer(x)
  loss = tf.reduce_mean(y**2)

# Calculate gradients with respect to every trainable variable
grad = tape.gradient(loss, layer.trainable_variables)

In [ ]:
for var, g in zip(layer.trainable_variables, grad):
  print(f'{var.name}, shape: {g.shape}')

<a id="watches"></a>

## Contrôler ce que regarde la bande

Le comportement par défaut est d'enregistrer toutes les opérations après avoir accédé à une `tf.Variable` . Les raisons en sont:

- La bande a besoin de savoir quelles opérations enregistrer dans la passe avant pour calculer les gradients dans la passe arrière.
- La bande contient des références aux sorties intermédiaires, vous ne voulez donc pas enregistrer d'opérations inutiles.
- Le cas d'utilisation le plus courant consiste à calculer le gradient d'une perte par rapport à toutes les variables entraînables d'un modèle.

For example the following fails to calculate a gradient because the `tf.Tensor` is not "watched" by default, and the `tf.Variable` is not trainable:

In [ ]:
# A trainable variable
x0 = tf.Variable(3.0, name='x0')
# Not trainable
x1 = tf.Variable(3.0, name='x1', trainable=False)
# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name='x2') + 1.0
# Not a variable
x3 = tf.constant(3.0, name='x3')

with tf.GradientTape() as tape:
  y = (x0**2) + (x1**2) + (x2**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
  print(g)

Vous pouvez lister les variables surveillées par la bande à l'aide de la méthode `GradientTape.watched_variables` :

In [ ]:
[var.name for var in tape.watched_variables()]

`tf.GradientTape` fournit des hooks qui donnent à l'utilisateur le contrôle de ce qui est ou non surveillé.

Pour enregistrer des dégradés par rapport à un `tf.Tensor` , vous devez appeler `GradientTape.watch(x)` :

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
  tape.watch(x)
  y = x**2

# dy = 2x * dx
dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())

Conversely, to disable the default behavior of watching all `tf.Variables`, set `watch_accessed_variables=False` when creating the gradient tape. This calculation uses two variables, but only connects the gradient for one of the variables:

In [ ]:
x0 = tf.Variable(0.0)
x1 = tf.Variable(10.0)

with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(x1)
  y0 = tf.math.sin(x0)
  y1 = tf.nn.softplus(x1)
  y = y0 + y1
  ys = tf.reduce_sum(y)

Puisque `GradientTape.watch` n'a pas été appelé sur `x0` , aucun gradient n'est calculé par rapport à celui-ci:

In [ ]:
# dy = 2x * dx
grad = tape.gradient(ys, {'x0': x0, 'x1': x1})

print('dy/dx0:', grad['x0'])
print('dy/dx1:', grad['x1'].numpy())

## Résultats intermédiaires

Vous pouvez également demander des dégradés de la sortie par rapport aux valeurs intermédiaires calculées dans le contexte `tf.GradientTape` .

In [ ]:
x = tf.constant(3.0)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = x * x
  z = y * y

# Use the tape to compute the gradient of z with respect to the
# intermediate value y.
# dz_dx = 2 * y, where y = x ** 2
print(tape.gradient(z, y).numpy())

Par défaut, les ressources détenues par un `GradientTape` sont libérées dès que la méthode `GradientTape.gradient()` est appelée. Pour calculer plusieurs dégradés sur le même calcul, créez une bande de dégradé `persistent` . Cela permet plusieurs appels à la méthode `gradient()` lorsque les ressources sont libérées lorsque l'objet bande est garbage collection. Par exemple:

In [ ]:
x = tf.constant([1, 3.0])
with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  y = x * x
  z = y * y

print(tape.gradient(z, x).numpy())  # 108.0 (4 * x**3 at x = 3)
print(tape.gradient(y, x).numpy())  # 6.0 (2 * x)

In [ ]:
del tape   # Drop the reference to the tape

## Notes sur les performances

- Il y a une minuscule surcharge associée à l'exécution d'opérations dans un contexte de bande dégradée. Pour les exécutions les plus pressantes, ce ne sera pas un coût notable, mais vous devez toujours utiliser le contexte de bande autour des zones uniquement là où il est nécessaire.

- Les bandes dégradées utilisent la mémoire pour stocker les résultats intermédiaires, y compris les entrées et les sorties, à utiliser pendant la passe arrière.

    Pour plus d'efficacité, certaines opérations (comme `ReLU` ) n'ont pas besoin de conserver leurs résultats intermédiaires et sont élaguées lors de la passe avant. Cependant, si vous utilisez `persistent=True` sur votre bande, *rien n'est supprimé* et votre utilisation maximale de la mémoire sera plus élevée.

## Dégradés de cibles non scalaires

Un gradient est fondamentalement une opération sur un scalaire.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient(y0, x).numpy())
print(tape.gradient(y1, x).numpy())

Ainsi, si vous demandez le gradient de plusieurs cibles, le résultat pour chaque source est:

- Le gradient de la somme des cibles, ou de manière équivalente
- La somme des gradients de chaque cible.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape() as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient({'y0': y0, 'y1': y1}, x).numpy())

De même, si la ou les cibles ne sont pas scalaires, le gradient de la somme est calculé:

In [ ]:
x = tf.Variable(2.)

with tf.GradientTape() as tape:
  y = x * [3., 4.]

print(tape.gradient(y, x).numpy())

Cela simplifie la prise du gradient de la somme d'une collection de pertes ou du gradient de la somme d'un calcul de perte élément par élément.

Si vous avez besoin d'un dégradé distinct pour chaque élément, consultez [Jacobiens](advanced_autodiff.ipynb#jacobians) .

Dans certains cas, vous pouvez ignorer le jacobien. Pour un calcul élément par élément, le gradient de la somme donne la dérivée de chaque élément par rapport à son élément d'entrée, puisque chaque élément est indépendant:

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = tf.nn.sigmoid(x)

dy_dx = tape.gradient(y, x)

In [ ]:
plt.plot(x, y, label='y')
plt.plot(x, dy_dx, label='dy/dx')
plt.legend()
_ = plt.xlabel('x')

## Contrôle du flux

Etant donné que les bandes enregistrent les opérations au fur et à mesure de leur exécution, le flux de contrôle Python (en utilisant `if` s et `while` s par exemple) est naturellement géré.

Ici, une variable différente est utilisée sur chaque branche d'un `if` . Le dégradé se connecte uniquement à la variable qui a été utilisée:

In [ ]:
x = tf.constant(1.0)

v0 = tf.Variable(2.0)
v1 = tf.Variable(2.0)

with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  if x > 0.0:
    result = v0
  else:
    result = v1**2 

dv0, dv1 = tape.gradient(result, [v0, v1])

print(dv0)
print(dv1)

Rappelez-vous simplement que les instructions de contrôle elles-mêmes ne sont pas différenciables, elles sont donc invisibles pour les optimiseurs basés sur le gradient.

Selon la valeur de `x` dans l'exemple ci-dessus, la bande enregistre le `result = v0` ou le `result = v1**2` . Le gradient par rapport à `x` est toujours `None` .

In [ ]:
dx = tape.gradient(result, x)

print(dx)

## Obtenir un dégradé de `None`

Lorsqu'une cible n'est pas connectée à une source, vous obtiendrez un dégradé de `None` .


In [ ]:
x = tf.Variable(2.)
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y * y
print(tape.gradient(z, x))

Ici, `z` n'est évidemment pas connecté à `x` , mais il existe plusieurs manières moins évidentes de déconnecter un gradient.

### 1. Remplacement d'une variable par un tenseur.

Dans la section sur ["contrôler ce que la bande regarde",](#watches) vous avez vu que la bande regardera automatiquement un `tf.Variable` mais pas un `tf.Tensor` .

Une erreur courante consiste à remplacer par inadvertance un `tf.Variable` par un `tf.Tensor` , au lieu d'utiliser `Variable.assign` pour mettre à jour le `tf.Variable` . Voici un exemple:

In [ ]:
x = tf.Variable(2.0)

for epoch in range(2):
  with tf.GradientTape() as tape:
    y = x+1

  print(type(x).__name__, ":", tape.gradient(y, x))
  x = x + 1   # This should be `x.assign_add(1)`

### 2. A fait des calculs en dehors de TensorFlow

La bande ne peut pas enregistrer le chemin du dégradé si le calcul quitte TensorFlow. Par exemple:

In [ ]:
x = tf.Variable([[1.0, 2.0],
                 [3.0, 4.0]], dtype=tf.float32)

with tf.GradientTape() as tape:
  x2 = x**2

  # This step is calculated with NumPy
  y = np.mean(x2, axis=0)

  # Like most ops, reduce_mean will cast the NumPy array to a constant tensor
  # using `tf.convert_to_tensor`.
  y = tf.reduce_mean(y, axis=0)

print(tape.gradient(y, x))

### 3. A pris des dégradés à travers un entier ou une chaîne

Les entiers et les chaînes ne sont pas différenciables. Si un chemin de calcul utilise ces types de données, il n'y aura pas de dégradé.

Personne ne s'attend à ce que les chaînes soient différenciables, mais il est facile de créer accidentellement une constante ou une variable `int` si vous ne spécifiez pas le `dtype` .

In [ ]:
# The x0 variable has an `int` dtype.
x = tf.Variable([[2, 2],
                 [2, 2]])

with tf.GradientTape() as tape:
  # The path to x1 is blocked by the `int` dtype here.
  y = tf.cast(x, tf.float32)
  y = tf.reduce_sum(x)

print(tape.gradient(y, x))

TensorFlow ne convertit pas automatiquement les types, donc en pratique, vous obtiendrez souvent une erreur de type au lieu d'un dégradé manquant.

### 5. Pris des dégradés à travers un objet avec état

L'état arrête les dégradés. Lorsque vous lisez à partir d'un objet avec état, la bande ne peut voir que l'état actuel, pas l'historique qui y mène.

Un `tf.Tensor` est immuable. Vous ne pouvez pas changer un tenseur une fois qu'il est créé. Il a une *valeur* , mais aucun *état* . Toutes les opérations décrites jusqu'ici sont également sans état: la sortie d'un `tf.matmul` ne dépend que de ses entrées.

Une `tf.Variable` a un état interne, sa valeur. Lorsque vous utilisez la variable, l'état est lu. Il est normal de calculer un gradient par rapport à une variable, mais l'état de la variable empêche les calculs de gradient de remonter plus loin. Par exemple:


In [ ]:
x0 = tf.Variable(3.0)
x1 = tf.Variable(0.0)

with tf.GradientTape() as tape:
  # Update x1 = x1 + x0.
  x1.assign_add(x0)
  # The tape starts recording from x1.
  y = x1**2   # y = (x1 + x0)**2

# This doesn't work.
print(tape.gradient(y, x0))   #dy/dx0 = 2*(x1 + x2)

De même, les itérateurs de `tf.data.Dataset` et de `tf.queue` sont avec état, et arrêteront tous les gradients sur les tenseurs qui les traversent.

## Aucun dégradé enregistré

Some `tf.Operation`s are **registered as being non-differentiable** and will return `None`. Others have **no gradient registered**.

La page [tf.raw_ops](https://www.tensorflow.org/api_docs/python/tf/raw_ops) montre quelles opérations de bas niveau ont des dégradés enregistrés.

Si vous essayez de prendre un dégradé à travers une opération float qui n'a pas de dégradé enregistré, la bande lèvera une erreur au lieu de renvoyer silencieusement `None` . De cette façon, vous savez que quelque chose ne va pas.

Par exemple, la fonction `tf.image.adjust_contrast` enveloppe [raw_ops.AdjustContrastv2](https://www.tensorflow.org/api_docs/python/tf/raw_ops#.AdjustContrastv2) qui pourrait avoir un dégradé mais le dégradé n'est pas implémenté:


In [ ]:
image = tf.Variable([[[0.5, 0.0, 0.0]]])
delta = tf.Variable(0.1)

with tf.GradientTape() as tape:
  new_image = tf.image.adjust_contrast(image, delta)

try:
  print(tape.gradient(new_image, [image, delta]))
  assert False   # This should not happen.
except LookupError as e:
  print(f'{type(e).__name__}: {e}')


Si vous avez besoin de différencier par cette opération, vous devrez soit implémenter le gradient et l'enregistrer (en utilisant `tf.RegisterGradient` ), soit ré-implémenter la fonction en utilisant d'autres opérations.

## Zéros au lieu de Aucun

Dans certains cas, il serait pratique d'obtenir 0 au lieu de `None` pour les dégradés non connectés. Vous pouvez décider quoi renvoyer lorsque vous avez des dégradés non connectés à l'aide de l'argument `unconnected_gradients` :

In [ ]:
x = tf.Variable([2., 2.])
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y**2
print(tape.gradient(z, x, unconnected_gradients=tf.UnconnectedGradients.ZERO))